<a href="https://colab.research.google.com/github/repitta/CienciaDeDadosEducacionais/blob/master/An%C3%A1lise_LopUsuarioNota.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importando as Bibliotecas**

In [0]:
import pandas as pd
from csv import reader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

**Lendo o arquivo CSV lopUsuarioNota**

In [0]:
# Ler o arquivo CSV 
lopNotasLista  = pd.read_csv("https://github.com/repitta/CienciaDeDadosEducacionais/raw/master/arquivosCSV/lopUsuarioNota.csv")
lopNotasProva  = pd.read_csv("https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lopUsuarioNotaProva.csv")

**Estrutura do aquivo**

In [0]:
print(lopNotasLista.info())
print(lopNotasLista.head())
print(lopNotasProva.info())
print(lopNotasProva.head())


**Pré-processamento**

In [0]:
##Tabela lopNotasLista
#excluir a coluna Unnamed
lopNotasLista = lopNotasLista[['titulo', 'idAluno', 'questoesFeitas', 'quantidade','NotaLista', 'ano', 'descricaoTurma', 'idTurma', 'matricula',"nome"]]
#dividir a coluna NotasLista por 10 , pq elas estão em porcentagem
lopNotasLista["NotaLista"] = lopNotasLista["NotaLista"] /10
#colocar letra maiuscula na coluna nome
lopNotasLista["nome"] = lopNotasLista["nome"].str.upper()

##Tabela lopNotasProva
#lopNotasProva = lopNotasProva.dropna(axis=1, how='all') 
lopNotasProva = lopNotasProva[['tituloProva', 'idAluno', 'questoesFeitasProva', 'quantidadeProva','notaProva', 'ano', 'descricaoTurma', 'idTurma', 'matricula',"nome"]]
#dividir a coluna NotasLista por 10 , pq elas estão em porcentagem
lopNotasProva["notaProva"] = lopNotasProva["notaProva"] /10
#colocar letra maiuscula na coluna nome
lopNotasProva["nome"] = lopNotasProva["nome"].str.upper()

**Funcao para formar a tabela de notas de todas as listas de exercicios**

In [0]:
def tabelaNotasLista(ano,turma):
  colunas    = ["matricula","nome"]
  tabelaListas = pd.DataFrame(columns=colunas)
  listas = ["Exercícios 5", "Exercícios 4.1", "Exercícios 4", "Exercícios 3", "Exercícios 2","Exercícios 1",
            "Laboratório 5","Laboratório 4", "Laboratório 3", "Laboratório 2","Laboratório 1"]
  for l in listas:
    notasTurma = lopNotasLista.loc[(lopNotasLista["ano"] == ano) & (lopNotasLista["descricaoTurma"].str.contains(turma, regex=False)) &
                 (lopNotasLista["titulo"].str.contains(l, regex=False)),["matricula","nome","NotaLista","questoesFeitas"]]
    notasTurma = notasTurma.sort_values("nome")
    tabelaListas = pd.merge(notasTurma,tabelaListas,on=['matricula','nome'], how="outer") 
    
  tabelaListas.columns = ['matricula', 'nome',"lab1","qLab1","lab2","qLab2","lab3","qLab3","lab4","qLab4","lab5","qLab5","exe1","qexe1","exe2","qexe2",
  "exe3","qexe3","exe4","qexe4","exe4.1","qexe4.1","exe5","qexe5"]
  tabelaListas = tabelaListas.drop_duplicates()
  tabelaListas = tabelaListas.sort_values('nome')
  return tabelaListas
  
tabelaLista = tabelaNotasLista(2019.1,"Turma 02D")
tabelaLista.to_excel(r'tabelaNotas2019Turma02D.xls')


**Função para formar a tabela das notas de todas as provas**

In [0]:
colunas    = ["matricula","nome"]
tabelaProvas = pd.DataFrame(columns=colunas)
ano = 2019.1
turma = "01D"
titulo = lopNotasProva.loc[(lopNotasProva["ano"] == ano)  & (lopNotasProva["descricaoTurma"].str.contains(turma, regex=False)),"tituloProva"].unique()
for l in titulo:
  notasProva =lopNotasProva.loc[(lopNotasProva["ano"] == ano)  & (lopNotasProva["descricaoTurma"].str.contains(turma, regex=False)) & (lopNotasProva["tituloProva"].str.contains(l, regex=False)),
                           ["matricula", "nome","notaProva","tituloProva", "quantidadeProva", "questoesFeitasProva"]]
  tabelaProvas = pd.concat([notasProva,tabelaProvas], ignore_index=True, axis = 0, sort=True)
tabelaProvas = tabelaProvas.sort_values("nome")
tabelaProvas.to_excel(r'tabelaNotasProva2019Turma01D.xls')

**Tabela de notas e quantidade de questões feitas por aluno por turma por lista**

In [0]:
notasTurma = lopNotas.loc[(lopNotas["ano"] == 2019.1) & (lopNotas["descricaoTurma"].str.contains("Turma 01A", regex=False)) &
                          (lopNotas["titulo"].str.contains("Lista de Exercícios 4", regex=False)),["nome","NotaLista","questoesFeitas"]]
notasTurma = notasTurma.sort_values("nome")
print("Quantidade de alunos que fizeram a lista:", notasTurma.shape[0])
print(notasTurma)


In [0]:
quanListAluno.loc[quanListAluno["titulo"].str.contains("Lista de Exercícios 1", regex=False),["ano","idAluno","totalAlunos"]]

In [0]:
quanListAluno.head()

**Gráfico para mostrar a quantidade de alunos que enviaram as listas ao longo dos anos**

In [0]:
#Contar quantos alunos submeteram cada lista ao longo do tempo
quanLista = lopNotas.groupby(['titulo','ano']).count()
quanLista = quanLista.reset_index()
#print(quanLista.head())

#contar quantos alunos estão cadastrados ao longo do tempo
c = lopNotas.groupby('ano')["idAluno"].unique()
c = c.reset_index()
alunoAno ={}
dtAlunoAno = pd.DataFrame()
for i in c["ano"].unique():
  q = c.loc[c["ano"] == i ,"idAluno"]
  alunoAno[i] = len(q.iloc[0])
dtAlunoAno["ano"] = alunoAno.keys()
dtAlunoAno["totalAlunos"] = alunoAno.values()
#dtAlunoAno

quanListAluno = pd.merge(dtAlunoAno,quanLista,on=['ano'], how="outer")
quanListAluno = quanListAluno.reset_index()

# Creating graphics with Seaborn
gQuandLista = sns.FacetGrid(quanListAluno, col = "titulo", col_wrap =2, height=3, aspect=2, sharex=False)
bp=gQuandLista.map(sns.barplot,"ano","idAluno" ,palette=sns.color_palette(['red','green']))

bp.add_legend()
bp.set_titles("{col_name}")
bp.set_ylabels("quantidade")
bp.set_xlabels("ano")
plt.show()


In [0]:
data=pd.DataFrame({'A':['X','X','Y','Y','X','X','Y','Y'],
               'B':[0,1,2,3,4,5,6,7],
               'C':[1,1,1,1,2,2,2,2],
               'type':['ctrl','cond1','ctrl','cond1','ctrl','cond1','ctrl','cond1']})
print(data.head(2))
print(quanListAluno.head(2))

**Gráfico para mostrar as médias das notas das listas ao longo do tempo**

In [0]:
mediaNota = lopNotas.groupby(['titulo','ano'])["NotaLista"].mean()
mediaNota = mediaNota.reset_index()
#print(mediaNota[["titulo",'NotaLista',"ano"]])

# Creating graphics with Seaborn
gMediaNota = sns.FacetGrid(mediaNota, col = "titulo", col_wrap=2, height=3, aspect=2, sharex=False)
gMediaNota.map(sns.barplot,"ano", "NotaLista")
plt.show()